**Fact Orders [Table Creation]**

Data Reading

In [0]:
df = spark.read.table("databricks_catalog.silver.orders_silver")
df.display()


In [0]:
df = spark.sql("select * from databricks_catalog.silver.orders_silver")
df.display()

In [0]:
df_dimcus = spark.sql("select DimCustomersKey, customer_id as Dim_customner_id from databricks_catalog.gold.dimcustomers")


df_dimpro = spark.sql("select product_id as DimProductsKey, product_id as Dim_product_key from databricks_catalog.gold.dimproducts")

**Fact Data Frame**

In [0]:
df_fact = df.join(df_dimcus, df["customer_id"] == df_dimcus["dim_customner_id"], how = "left").join(df_dimpro, df["product_id"] == df_dimpro["Dim_product_key"], how = "left")
                                                                                                  
df_fact_new = df_fact.drop("dim_customner_id", "Dim_product_key","customer_id","product_id")
df_fact_new.display()                                                                                                
                                                                

**apply upsert on fact table**

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("databricks_catalog.gold.FactOrders"):
    dlt_obj = DeltaTable.forName(spark, "databricks_catalog.gold.FactOrders")

    dlt_obj.alias("trg")\
          .merge(
            df_fact_new.alias("src"), 
            "trg.order_id = src.order_id AND trg.DimCustomersKey = src.DimCustomersKey AND trg.DimProductsKey = src.DimProductsKey"
          ) \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    df_fact_new.write.format("delta")\
      .option("path", "abfss://gold@ssdatalakeeteproject.dfs.core.windows.net/FactOrders")\
      .saveAsTable("databricks_catalog.gold.FactOrders")

In [0]:
%sql
select * from databricks_catalog.gold.FactOrders